In [1]:
import pandas as pd

dataset = pd.read_csv('cluster_3.csv')
print(dataset.head())

   satisfaction_level  last_evaluation  number_project  average_montly_hours  \
0                0.10             0.77               6                   247   
1                0.11             0.81               6                   305   
2                0.11             0.83               6                   282   
3                0.09             0.95               6                   304   
4                0.09             0.62               6                   294   

   time_spend_company  Work_accident  left  promotion_last_5years  sales  \
0                   4              0     1                      0      7   
1                   4              0     1                      0      7   
2                   4              0     1                      0      7   
3                   4              0     1                      0      7   
4                   4              0     1                      0      2   

   salary  cluster  
0       1        3  
1       1        3  

In [2]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

dataset[dataset.drop(['cluster', 'salary', 'sales', 'promotion_last_5years', 'left', 'Work_accident'], axis=1).columns] = scaler.fit_transform(
    dataset[dataset.drop(['cluster', 'salary', 'sales', 'promotion_last_5years', 'left', 'Work_accident'], axis=1).columns]
)

print(dataset.head())

   satisfaction_level  last_evaluation  number_project  average_montly_hours  \
0            0.018519         0.640625            0.75              0.705607   
1            0.037037         0.703125            0.75              0.976636   
2            0.037037         0.734375            0.75              0.869159   
3            0.000000         0.921875            0.75              0.971963   
4            0.000000         0.406250            0.75              0.925234   

   time_spend_company  Work_accident  left  promotion_last_5years  sales  \
0            0.333333              0     1                      0      7   
1            0.333333              0     1                      0      7   
2            0.333333              0     1                      0      7   
3            0.333333              0     1                      0      7   
4            0.333333              0     1                      0      2   

   salary  cluster  
0       1        3  
1       1        3  

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = dataset.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlation Matrix")
plt.show()

In [3]:
x = dataset.drop(['left', 'cluster'], axis=1)
y = dataset['left']

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score

knn_model = KNeighborsClassifier()
knn_model.fit(x_train, y_train)

y_pred_knn = knn_model.predict(x_test)

print(f'KNN Results:\nAccuracy: {accuracy_score(y_test, y_pred_knn)}\nRecall: {recall_score(y_test, y_pred_knn)}')

KNN Results:
Accuracy: 0.8652173913043478
Recall: 0.8571428571428571


In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score

svm = SVC(kernel='linear')
svm.fit(x_train, y_train)

y_pred_svm = svm.predict(x_test)

print(f'SVM Results:\nAccuracy: {accuracy_score(y_test, y_pred_svm)}\nRecall: {recall_score(y_test, y_pred_svm)}')

SVM Results:
Accuracy: 0.9260869565217391
Recall: 0.8809523809523809


In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, recall_score

# Naive Bayes model
nb = GaussianNB()
nb.fit(x_train, y_train)

# Predictions
y_pred_nb = nb.predict(x_test)

print(f'NB Results:\nAccuracy: {accuracy_score(y_test, y_pred_nb)}\nRecall: {recall_score(y_test, y_pred_nb)}')

NB Results:
Accuracy: 0.8782608695652174
Recall: 0.8095238095238095


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def create_ann(units, activation, optimizer, learning_rate, batch_size, weight_decay, hidden_layers):
    model = Sequential()
    
    # Add first hidden layer
    model.add(Dense(units=units, activation=activation, input_dim=x_train.shape[1]))
    
    # Add additional hidden layers
    for _ in range(hidden_layers - 1):
        model.add(Dense(units=units, activation=activation))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))  # For binary classification
    
    # Compile the model
    optimizer_instance = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer_instance, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
params = {'units': 32, 'activation': 'relu', 'optimizer': 'adam', 'learning_rate': 0.005127, 
                  'batch_size': 32, 'weight_decay': 0.000027, 'hidden_layers': 5, 'epochs': 150}

model = create_ann(
        units=params['units'],
        activation=params['activation'],
        optimizer=params['optimizer'],
        learning_rate=params['learning_rate'],
        batch_size=params['batch_size'],
        weight_decay=params['weight_decay'],
        hidden_layers=params['hidden_layers']
    )

c:\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.fit(x_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'], verbose=0)

In [11]:
# Evaluate the model on the test data
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert predictions to binary labels

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [12]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"\nResults:")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{classification_rep}")


Results:
Accuracy: 0.9391304347826087
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.99      0.94       104
           1       0.99      0.90      0.94       126

    accuracy                           0.94       230
   macro avg       0.94      0.94      0.94       230
weighted avg       0.94      0.94      0.94       230



In [13]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE for class imbalance (if necessary)
smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(x, y)

# Create DataFrames with consistent indices
x_resampled_df = pd.DataFrame(x_resampled, columns=x.columns)  # Resampled features
y_resampled_df = pd.Series(y_resampled, name='left')  # Resampled target

# Combine the resampled features and target
augmented_dataset = pd.concat([x_resampled_df, y_resampled_df], axis=1)

In [14]:
print(augmented_dataset.groupby('left').count())

      satisfaction_level  last_evaluation  number_project  \
left                                                        
0                    593              593             593   
1                    593              593             593   

      average_montly_hours  time_spend_company  Work_accident  \
left                                                            
0                      593                 593            593   
1                      593                 593            593   

      promotion_last_5years  sales  salary  
left                                        
0                       593    593     593  
1                       593    593     593  


In [15]:
x = augmented_dataset.drop(['left'], axis=1)
y = augmented_dataset['left']

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [17]:
model.fit(x_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'], verbose=0)

In [18]:
# Evaluate the model on the test data
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert predictions to binary labels

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 861us/step


In [19]:
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"\nResults After Augmentation:")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{classification_rep}")


Results After Augmentation:
Accuracy: 0.9621848739495799
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.96       121
           1       0.97      0.95      0.96       117

    accuracy                           0.96       238
   macro avg       0.96      0.96      0.96       238
weighted avg       0.96      0.96      0.96       238

